## 10. Collaboration

### 85 Use Packages to Organize Modules and Provide Stable APIs

#### 이름 공간

```python
# analysis/utils/utils.py

import math

def log_base2_bucket(value):
    return math.log(value, 2)

def inspect(value):
    pass
```

```python
# analysis/utils/__init__.py
```

```python
# frontend/utils/utils.py

def stringify(value):
    return str(value)

def inspect(value):
    pass

```

```python
# frontend/utils/__init__.py
```

```python
# main1.py

from analysis.utils import log_base2_bucket
from frontend.utils import stringify

bucket = stringify(log_base2_bucket(33))
print(repr(bucket))
```

```shell
$ python main1.py 
'5.044394119358453'

```

```python
# main2.py

from analysis.utils import inspect
from frontend.utils import inspect  # Overwrites!
'frontend' in inspect.__module__
print(inspect.__module__)
```

```shell
$ python main2.py 
frontend.utils
```

```python
# main3.py

from analysis.utils import inspect as analysis_inspect
from frontend.utils import inspect as frontend_inspect

value = 33
if analysis_inspect(value) == frontend_inspect(value):
    print('Inspection equal!')
```

```shell
$ python main3.py 
Inspection equal!
```

```python
# main4.py

import analysis.utils
import frontend.utils

value = 33
if (analysis.utils.inspect(value) == frontend.utils.inspect(value)):
    print('Inspection equal!')
```

```shell
$ python main4.py 
Inspection equal!
```

#### 안정적인 API

```python
# mypackage/models.py

__all__ = ['Projectile']

class Projectile:
    def __init__(self, mass, velocity):
        self.mass = mass
        self.velocity = velocity
```

```python
# mypackage/utils.py

from .models import Projectile

__all__ = ['simulate_collision']

def _dot_product(a, b):
    pass

def simulate_collision(a, b):
    after_a = Projectile(-a.mass, -a.velocity)
    after_b = Projectile(-b.mass, -b.velocity)
    return after_a, after_b
```

```python
# mypackage/__init__.py

__all__ = []
from .models import *
__all__ += models.__all__
from .utils import *
__all__ += utils.__all__
```

```python
# api_consumer.py

from mypackage import *

a = Projectile(1.5, 3)
b = Projectile(4, 1.7)
after_a, after_b = simulate_collision(a, b)
print(after_a.__dict__, after_b.__dict__)
```

```shell
$ python api_consumer.py 
{'mass': -1.5, 'velocity': -3} {'mass': -4, 'velocity': -1.7}
```

```shell
$ python
>>> from mypackage import utils
>>> utils._dot_product
<function _dot_product at 0x7f7e323ca5e0>
>>> 
>>> import mypackage
>>> try:
...     mypackage._dot_product
...     assert False
... except AttributeError:
...     pass  # Expected
... 
>>> mypackage.utils._dot_product
<function _dot_product at 0x7f7e323ca5e0>
```

> - 파이썬 패키지는 다른 모듈을 포함하는 모듈이다. 패키지를 사용하면 서로 분리돼 충돌이 일어나지 않는, 유일한 절대 모듈 경로를 사용하는 이름 공간으로 코드를 나눌 수 있다.
> - 다른 소스 파일이 들어 있는 디렉토리에 `__init__.py` 파일을 추가하면 간단한 패키지를 만들 수 있다. 소스 파일들은 디렉토리로 인해 생긴 패키지의 자식 모듈이 된다. 패키지 디렉토리에는 다른 패키지가 들어갈 수도 있다.
> - 모듈 외부에서 볼 수 있게 허용할 이름들을 `__all__` 특별 애트리뷰트에 지정해 공개 API를 제공할 수 있다.
> - 패키지의 `__init__.py` 파일에 외부에 공개할 이름만 임포트하거나 패키지 내부에서만 사용할 이름 앞에 `_`를 붙임으로써 패키지 내부에서만 사용할 수 있는 이름을 감출 수 있다.
> - 단일 코드베이스나 단일 팀 안에서 협업을 진행한다면 아마도 `__all__`로 API를 명시할 필요가 없을 것이다.

> - `from x import *` 스타일로 `import *` 문을 여러 번 사용하면 어떤 이름이 어느 모듈에서 비롯됐는지 알기 어렵고, 이름이 겹치면 기존 이름을 덮어 쓰게 된다.
> - 안전한 접근 방법은 명시적으로 이름을 임포트하는 것이다.
>   - `from x import y`
>   - `import x`  
>     `x.y`